# HAR Data Fusion Based Algorithm #

In [1]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import bnlearn as bn
from sklearn.model_selection import StratifiedKFold

2024-01-30 16:12:08.954023: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Lista delle Attività "semanticamente semplici" e degli Utenti ##

In [2]:
def getList(dirName):
    listofF = os.listdir(dirName)
    allFiles = list()

    for e in listofF:

        fullpat = os.path.join(dirName, e)
        if os.path.isdir(fullpat):
            allFiles = allFiles + getList(fullpat)
        else:
            if re.search(".*", e):
                allFiles.append(fullpat)
    return allFiles

In [3]:
listaUtenti = ["Giuseppe", "Maria", "Michele", "Andrea", "Alessio", "Marco", "Salvo", "Angelo"]

listaAttivita = ["Seduto", "Camminata", "Corsa", "Salire", "Scendere"]

## Creazione dei Dizionari ##

In [4]:
dizionarioDatiTuttiUtentiMaster = dict()

for utente in listaUtenti:
    # Inizializza il dizionario per l'utente corrente
    dizionarioDatiTuttiUtentiMaster[utente] = {}  
    for attivita in listaAttivita:
        # Ottieni la lista di file per l'utente e l'attività correnti
        listaDati = getList('../risultati' + "/" + utente + "/" + attivita)
        
        # Filtra solo i percorsi che contengono "StatisticheMasterDati" nel nome
        listaDati = [path for path in listaDati if "StatisticheMasterPhased" in path]
        
        # Aggiungi la lista di dati al dizionario
        dizionarioDatiTuttiUtentiMaster[utente][attivita] = listaDati

# Stampa il dizionario risultante
for utente, attivita_dict in dizionarioDatiTuttiUtentiMaster.items():
    print(f"Utente: {utente}")
    for attivita, listaDati in attivita_dict.items():
        print(f"\tAttività: {attivita}")
        for file_path in listaDati:
            print(f"\t\t{file_path}")

Utente: Giuseppe
	Attività: Seduto
		../risultati/Giuseppe/Seduto/StatisticheMasterPhased_Seduto2024_01_18_17_18_06.csv
		../risultati/Giuseppe/Seduto/StatisticheMasterPhased_Seduto2024_01_18_17_18_04.csv
	Attività: Camminata
		../risultati/Giuseppe/Camminata/StatisticheMasterPhased_Camminata2023_04_08_17_23_33.csv
		../risultati/Giuseppe/Camminata/StatisticheMasterPhased_Camminata2023_04_08_16_44_07.csv
		../risultati/Giuseppe/Camminata/StatisticheMasterPhased_Camminata2023_04_08_16_58_04.csv
	Attività: Corsa
		../risultati/Giuseppe/Corsa/StatisticheMasterPhased_Corsa2023_04_08_17_03_06.csv
		../risultati/Giuseppe/Corsa/StatisticheMasterPhased_Corsa2023_04_08_17_48_44.csv
	Attività: Salire
		../risultati/Giuseppe/Salire/StatisticheMasterPhased_Salire2023_04_08_16_53_05.csv
		../risultati/Giuseppe/Salire/StatisticheMasterPhased_Salire2023_04_08_16_48_40.csv
		../risultati/Giuseppe/Salire/StatisticheMasterPhased_Salire2023_04_08_16_55_11.csv
		../risultati/Giuseppe/Salire/StatisticheMas

In [5]:
dizionarioDatiTuttiUtentiSlave = dict()

for utente in listaUtenti:
    # Inizializza il dizionario per l'utente corrente
    dizionarioDatiTuttiUtentiSlave[utente] = {}  
    for attivita in listaAttivita:
        # Ottieni la lista di file per l'utente e l'attività correnti
        listaDati = getList('../risultati' + "/" + utente + "/" + attivita)
        
        # Filtra solo i percorsi che contengono "StatisticheSlaveDati" nel nome
        listaDati = [path for path in listaDati if "StatisticheSlavePhased" in path]
        
        # Aggiungi la lista di dati al dizionario
        dizionarioDatiTuttiUtentiSlave[utente][attivita] = listaDati

# Stampa il dizionario risultante
for utente, attivita_dict in dizionarioDatiTuttiUtentiSlave.items():
    print(f"Utente: {utente}")
    for attivita, listaDati in attivita_dict.items():
        print(f"\tAttività: {attivita}")
        for file_path in listaDati:
            print(f"\t\t{file_path}")

Utente: Giuseppe
	Attività: Seduto
		../risultati/Giuseppe/Seduto/StatisticheSlavePhased_Seduto2024_01_18_17_18_04.csv
		../risultati/Giuseppe/Seduto/StatisticheSlavePhased_Seduto2024_01_18_17_18_06.csv
	Attività: Camminata
		../risultati/Giuseppe/Camminata/StatisticheSlavePhased_Camminata2023_04_08_17_23_33.csv
		../risultati/Giuseppe/Camminata/StatisticheSlavePhased_Camminata2023_04_08_16_44_07.csv
		../risultati/Giuseppe/Camminata/StatisticheSlavePhased_Camminata2023_04_08_16_58_04.csv
	Attività: Corsa
		../risultati/Giuseppe/Corsa/StatisticheSlavePhased_Corsa2023_04_08_17_48_44.csv
		../risultati/Giuseppe/Corsa/StatisticheSlavePhased_Corsa2023_04_08_17_03_06.csv
	Attività: Salire
		../risultati/Giuseppe/Salire/StatisticheSlavePhased_Salire2023_04_08_16_53_05.csv
		../risultati/Giuseppe/Salire/StatisticheSlavePhased_Salire2023_04_08_16_55_11.csv
		../risultati/Giuseppe/Salire/StatisticheSlavePhased_Salire2023_04_08_16_48_40.csv
		../risultati/Giuseppe/Salire/StatisticheSlavePhased_S

## Creazione dei Dataset delle Features ##

In [6]:
# Dizionario principale che conterrà i risultati per ogni configurazione
risultati_configurazione_master = {}

for utente in listaUtenti:
    # DataFrame per l'utente corrente
    df_tot_master = pd.DataFrame()

    for attivita in listaAttivita:
        # Filtra solo i percorsi che terminano con ".csv"
        csv_paths = [path for path in dizionarioDatiTuttiUtentiMaster[utente][attivita] if path.endswith('.csv')]

        for csv_path in csv_paths:
            df = pd.read_csv(csv_path)
                    
            # Sostituisci 'Alzato' o 'Seduto' con 'Fermo'
            df['Attivita'] = df['Attivita'].replace(['Seduto'], 'Fermo')
                    
            df_tot_master = pd.concat([df, df_tot_master], ignore_index=True)
            
        # Controllo se il DataFrame non è vuoto prima di aggiungerlo al dizionario della configurazione
        if not df_tot_master.empty:
            risultati_configurazione_master[utente] = df_tot_master

for utente, df_tot_master in risultati_configurazione_master.items():
    print(f"User {utente}:")
    print(df_tot_master)
    print(df_tot_master.shape)

User Giuseppe:
       Attivita FinestraID DimensioneFinestra Overlap  AccelerometroXMedia  \
0      Scendere          0                 10       9            -1.674223   
1      Scendere          1                 10       9            -1.336671   
2      Scendere          2                 10       9            -0.967084   
3      Scendere          3                 10       9            -0.304570   
4      Scendere          4                 10       9            -1.960189   
...         ...        ...                ...     ...                  ...   
22831     Fermo       1033                 10       9             4.175103   
22832     Fermo       1034                 10       9             4.181987   
22833     Fermo       1035                 10       9             4.174371   
22834     Fermo       1036                 10       9             4.184318   
22835     Fermo       1037                 10       9             4.191141   

       AccelerometroXDeviazioneStandard  Acceler

In [7]:
# Dizionario principale che conterrà i risultati per ogni configurazione
risultati_configurazione_slave = {}

for utente in listaUtenti:
    # DataFrame per l'utente corrente
    df_tot_slave = pd.DataFrame()

    for attivita in listaAttivita:
        # Filtra solo i percorsi che terminano con ".csv"
        csv_paths = [path for path in dizionarioDatiTuttiUtentiSlave[utente][attivita] if path.endswith('.csv')]

        for csv_path in csv_paths:
            df = pd.read_csv(csv_path)
                    
            # Sostituisci 'Alzato' o 'Seduto' con 'Fermo'
            df['Attivita'] = df['Attivita'].replace(['Seduto'], 'Fermo')
                    
            df_tot_slave = pd.concat([df, df_tot_slave], ignore_index=True)
            
        # Controllo se il DataFrame non è vuoto prima di aggiungerlo al dizionario della configurazione
        if not df_tot_slave.empty:
            risultati_configurazione_slave[utente] = df_tot_slave

for utente, df_tot_slave in risultati_configurazione_slave.items():
    print(f"User {utente}:")
    print(df_tot_slave)
    print(df_tot_slave.shape)

User Giuseppe:
       Attivita FinestraID DimensioneFinestra Overlap  AccelerometroXMedia  \
0      Scendere          0                 10       9             0.686183   
1      Scendere          1                 10       9             0.648676   
2      Scendere          2                 10       9             0.650812   
3      Scendere          3                 10       9             0.813183   
4      Scendere          4                 10       9             0.837814   
...         ...        ...                ...     ...                  ...   
22858     Fermo       1031                 10       9             7.539166   
22859     Fermo       1032                 10       9             7.544235   
22860     Fermo       1033                 10       9             7.528700   
22861     Fermo       1034                 10       9             7.546069   
22862     Fermo       1035                 10       9             7.564268   

       AccelerometroXDeviazioneStandard  Acceler

## Bilanciamento dei Dataset delle Features ##

In [8]:
listaAttivita = ["Fermo", "Camminata", "Corsa", "Salire", "Scendere"]

In [9]:
featureGlobalMaster = {}
featureGlobalMasterBilanciato = {}

# Itera attraverso i risultati della configurazione master
for utente, df_tot_master in risultati_configurazione_master.items():
    attivitaMaster = df_tot_master['Attivita']
    finestraIDMaster = df_tot_master['FinestraID']
    featuresMaster = df_tot_master.iloc[:, 4:]

    # Aggiungi la colonna 'Attivita' al dataframe delle features
    featuresMaster['Attivita'] = attivitaMaster
    featuresMaster['FinestraID'] = finestraIDMaster

    # Aggiungi i risultati della configurazione all'interno del dizionario dell'utente
    featureGlobalMaster[utente] = featuresMaster

# Itera attraverso il dizionario di utenti e dataframe
for utente, featuresMaster in featureGlobalMaster.items():
    print(f"Utente: {utente}")
    
    # Trova il numero minimo di occorrenze per ciascuna classe
    min_occorrenze_classe = featuresMaster['Attivita'].value_counts().min()

    # Tronca tutti i dataframe dell'utente al numero minimo di occorrenze
    featuresMaster_troncati = featuresMaster.groupby('Attivita').sample(n=min_occorrenze_classe, random_state=42)
    
    print(featuresMaster_troncati['Attivita'].value_counts())

    featureGlobalMasterBilanciato[utente] = featuresMaster_troncati

Utente: Giuseppe
Camminata    713
Corsa        713
Fermo        713
Salire       713
Scendere     713
Name: Attivita, dtype: int64
Utente: Maria
Camminata    545
Corsa        545
Fermo        545
Salire       545
Scendere     545
Name: Attivita, dtype: int64
Utente: Michele
Camminata    860
Corsa        860
Fermo        860
Salire       860
Scendere     860
Name: Attivita, dtype: int64
Utente: Andrea
Camminata    767
Corsa        767
Fermo        767
Salire       767
Scendere     767
Name: Attivita, dtype: int64
Utente: Alessio
Camminata    869
Corsa        869
Fermo        869
Salire       869
Scendere     869
Name: Attivita, dtype: int64
Utente: Marco
Camminata    902
Corsa        902
Fermo        902
Salire       902
Scendere     902
Name: Attivita, dtype: int64
Utente: Salvo
Camminata    868
Corsa        868
Fermo        868
Salire       868
Scendere     868
Name: Attivita, dtype: int64
Utente: Angelo
Camminata    773
Corsa        773
Fermo        773
Salire       773
Scendere     

In [10]:
featureGlobalSlave = {}
featureGlobalSlaveBilanciato = {}

# Itera attraverso i risultati della configurazione master
for utente, df_tot_slave in risultati_configurazione_slave.items():
    attivitaSlave = df_tot_slave['Attivita']
    finestraIDSlave = df_tot_slave['FinestraID']
    featuresSlave = df_tot_slave.iloc[:, 4:]

    # Aggiungi la colonna 'Attivita' al dataframe delle features
    featuresSlave['Attivita'] = attivitaSlave
    featuresSlave['FinestraID'] = finestraIDSlave

    # Aggiungi i risultati della configurazione all'interno del dizionario dell'utente
    featureGlobalSlave[utente] = featuresSlave

# Itera attraverso il dizionario di utenti e dataframe
for utente, featuresSlave in featureGlobalSlave.items():
    print(f"Utente: {utente}")
    
    # Trova il numero minimo di occorrenze per ciascuna classe
    min_occorrenze_classe = featuresSlave['Attivita'].value_counts().min()

    # Tronca tutti i dataframe dell'utente al numero minimo di occorrenze
    featuresSlave_troncati = featuresSlave.groupby('Attivita').sample(n=min_occorrenze_classe, random_state=42)
    
    print(featuresSlave_troncati['Attivita'].value_counts())

    featureGlobalSlaveBilanciato[utente] = featuresSlave_troncati

Utente: Giuseppe
Camminata    712
Corsa        712
Fermo        712
Salire       712
Scendere     712
Name: Attivita, dtype: int64
Utente: Maria
Camminata    546
Corsa        546
Fermo        546
Salire       546
Scendere     546
Name: Attivita, dtype: int64
Utente: Michele
Camminata    861
Corsa        861
Fermo        861
Salire       861
Scendere     861
Name: Attivita, dtype: int64
Utente: Andrea
Camminata    778
Corsa        778
Fermo        778
Salire       778
Scendere     778
Name: Attivita, dtype: int64
Utente: Alessio
Camminata    871
Corsa        871
Fermo        871
Salire       871
Scendere     871
Name: Attivita, dtype: int64
Utente: Marco
Camminata    902
Corsa        902
Fermo        902
Salire       902
Scendere     902
Name: Attivita, dtype: int64
Utente: Salvo
Camminata    870
Corsa        870
Fermo        870
Salire       870
Scendere     870
Name: Attivita, dtype: int64
Utente: Angelo
Camminata    772
Corsa        772
Fermo        772
Salire       772
Scendere     

In [11]:
df_features_master = pd.DataFrame()
    
# Concatenate the features for each user
for utente, featuresMaster_troncati in featureGlobalMasterBilanciato.items():
        
    df_features_master = pd.concat([df_features_master, featuresMaster_troncati], ignore_index=True)

print(df_features_master.shape)

(31485, 37)


In [12]:
df_features_slave = pd.DataFrame()
    
# Concatenate the features for each user
for utente, featuresSlave_troncati in featureGlobalSlaveBilanciato.items():
        
    df_features_slave = pd.concat([df_features_slave, featuresSlave_troncati], ignore_index=True)

print(df_features_slave.shape)

(31560, 37)


In [15]:
# Esempio: Conteggio delle attività in df_features_master
activity_counts_master = df_features_master['Attivita'].value_counts()

# Esempio: Conteggio delle attività nello slave DataFrame
activity_counts_slave = df_features_slave['Attivita'].value_counts()

# Trova l'attività con il conteggio minore tra master e slave
min_activity_count = min(activity_counts_master.min(), activity_counts_slave.min())

# Filtra le attività nel master DataFrame in modo che ciascuna abbia lo stesso conteggio minimo
df_features_master_truncated = df_features_master.groupby('Attivita').sample(n=min_activity_count, random_state=42)

df_features_slave_truncated = df_features_slave.groupby('Attivita').sample(n=min_activity_count, random_state=42)

## Train-Test Split ##

In [18]:
features_train_master, features_test_master = train_test_split(df_features_master_truncated, test_size=0.2, random_state=42, stratify=df_features_master_truncated['Attivita'])
features_train_slave, features_test_slave = train_test_split(df_features_slave_truncated, test_size=0.2, random_state=42, stratify=df_features_slave_truncated['Attivita'])

In [19]:
print(features_train_master['Attivita'].value_counts())
print(features_train_slave['Attivita'].value_counts())

Fermo        5038
Scendere     5038
Corsa        5038
Camminata    5037
Salire       5037
Name: Attivita, dtype: int64
Fermo        5038
Scendere     5038
Corsa        5038
Camminata    5037
Salire       5037
Name: Attivita, dtype: int64


In [20]:
print(features_test_master['Attivita'].value_counts())
print(features_test_slave['Attivita'].value_counts())

Salire       1260
Camminata    1260
Fermo        1259
Corsa        1259
Scendere     1259
Name: Attivita, dtype: int64
Salire       1260
Camminata    1260
Fermo        1259
Corsa        1259
Scendere     1259
Name: Attivita, dtype: int64


## Lista delle Attività "semanticamente complesse" e degli Utenti ##

In [21]:
listaAttivita2 = ["Guidare", "StareAlPc"]
listaUtenti2 = ["Giuseppe"]

## Creazione dei Dizionari ##

In [22]:
dizionarioDatiTuttiUtentiMaster = dict()

for utente in listaUtenti2:
    # Inizializza il dizionario per l'utente corrente
    dizionarioDatiTuttiUtentiMaster[utente] = {}  
    for attivita in listaAttivita2:
        # Ottieni la lista di file per l'utente e l'attività correnti
        listaDati = getList('../risultati' + "/" + utente + "/" + attivita)
        
        # Filtra solo i percorsi che contengono "StatisticheMasterDati" nel nome
        listaDati = [path for path in listaDati if "StatisticheMasterPhased" in path]
        
        # Aggiungi la lista di dati al dizionario
        dizionarioDatiTuttiUtentiMaster[utente][attivita] = listaDati

# Stampa il dizionario risultante
for utente, attivita_dict in dizionarioDatiTuttiUtentiMaster.items():
    print(f"Utente: {utente}")
    for attivita, listaDati in attivita_dict.items():
        print(f"\tAttività: {attivita}")
        for file_path in listaDati:
            print(f"\t\t{file_path}")

Utente: Giuseppe
	Attività: Guidare
		../risultati/Giuseppe/Guidare/StatisticheMasterPhased_Guidare2024_01_30_13_26_00.csv
		../risultati/Giuseppe/Guidare/StatisticheMasterPhased_Guidare2024_01_18_18_08_16.csv
	Attività: StareAlPc
		../risultati/Giuseppe/StareAlPc/StatisticheMasterPhased_StareAlPc2024_01_18_13_20_49.csv


In [23]:
dizionarioDatiTuttiUtentiSlave = dict()

for utente in listaUtenti2:
    # Inizializza il dizionario per l'utente corrente
    dizionarioDatiTuttiUtentiSlave[utente] = {}  
    for attivita in listaAttivita2:
        # Ottieni la lista di file per l'utente e l'attività correnti
        listaDati = getList('../risultati' + "/" + utente + "/" + attivita)
        
        # Filtra solo i percorsi che contengono "StatisticheSlaveDati" nel nome
        listaDati = [path for path in listaDati if "StatisticheSlavePhased" in path]
        
        # Aggiungi la lista di dati al dizionario
        dizionarioDatiTuttiUtentiSlave[utente][attivita] = listaDati

# Stampa il dizionario risultante
for utente, attivita_dict in dizionarioDatiTuttiUtentiSlave.items():
    print(f"Utente: {utente}")
    for attivita, listaDati in attivita_dict.items():
        print(f"\tAttività: {attivita}")
        for file_path in listaDati:
            print(f"\t\t{file_path}")

Utente: Giuseppe
	Attività: Guidare
		../risultati/Giuseppe/Guidare/StatisticheSlavePhased_Guidare2024_01_30_13_26_00.csv
		../risultati/Giuseppe/Guidare/StatisticheSlavePhased_Guidare2024_01_18_18_08_16.csv
	Attività: StareAlPc
		../risultati/Giuseppe/StareAlPc/StatisticheSlavePhased_StareAlPc2024_01_18_13_20_49.csv


## Creazione dei Dataset delle Features ##

In [24]:
# Dizionario principale che conterrà i risultati per ogni configurazione
risultati_configurazione_master = {}

for utente in listaUtenti2:
    # DataFrame per l'utente corrente
    df_tot_master = pd.DataFrame()

    for attivita in listaAttivita2:
        # Filtra solo i percorsi che terminano con ".csv"
        csv_paths = [path for path in dizionarioDatiTuttiUtentiMaster[utente][attivita] if path.endswith('.csv')]

        for csv_path in csv_paths:
            df = pd.read_csv(csv_path)
                    
            # Sostituisci 'Alzato' o 'Seduto' con 'Fermo'
            df['Attivita'] = df['Attivita'].replace(['Seduto'], 'Fermo')
                    
            df_tot_master = pd.concat([df, df_tot_master], ignore_index=True)
            
        # Controllo se il DataFrame non è vuoto prima di aggiungerlo al dizionario della configurazione
        if not df_tot_master.empty:
            risultati_configurazione_master[utente] = df_tot_master

for utente, df_tot_master in risultati_configurazione_master.items():
    print(f"User {utente}:")
    print(df_tot_master)
    print(df_tot_master.shape)

User Giuseppe:
        Attivita  FinestraID  DimensioneFinestra  Overlap  \
0      StareAlPc           0                  10        9   
1      StareAlPc           1                  10        9   
2      StareAlPc           2                  10        9   
3      StareAlPc           3                  10        9   
4      StareAlPc           4                  10        9   
...          ...         ...                 ...      ...   
15238    Guidare        3954                  10        9   
15239    Guidare        3955                  10        9   
15240    Guidare        3956                  10        9   
15241    Guidare        3957                  10        9   
15242    Guidare        3958                  10        9   

       AccelerometroXMedia  AccelerometroXDeviazioneStandard  \
0                 3.381164                          1.378078   
1                 3.405074                          1.361346   
2                 3.261795                          1.192537

In [25]:
# Dizionario principale che conterrà i risultati per ogni configurazione
risultati_configurazione_slave = {}

for utente in listaUtenti2:
    # DataFrame per l'utente corrente
    df_tot_slave = pd.DataFrame()

    for attivita in listaAttivita2:
        # Filtra solo i percorsi che terminano con ".csv"
        csv_paths = [path for path in dizionarioDatiTuttiUtentiSlave[utente][attivita] if path.endswith('.csv')]

        for csv_path in csv_paths:
            df = pd.read_csv(csv_path)
                    
            # Sostituisci 'Alzato' o 'Seduto' con 'Fermo'
            df['Attivita'] = df['Attivita'].replace(['Seduto'], 'Fermo')
                    
            df_tot_slave = pd.concat([df, df_tot_slave], ignore_index=True)
            
        # Controllo se il DataFrame non è vuoto prima di aggiungerlo al dizionario della configurazione
        if not df_tot_slave.empty:
            risultati_configurazione_slave[utente] = df_tot_slave

for utente, df_tot_slave in risultati_configurazione_slave.items():
    print(f"User {utente}:")
    print(df_tot_slave)
    print(df_tot_slave.shape)

User Giuseppe:
        Attivita  FinestraID  DimensioneFinestra  Overlap  \
0      StareAlPc           0                  10        9   
1      StareAlPc           1                  10        9   
2      StareAlPc           2                  10        9   
3      StareAlPc           3                  10        9   
4      StareAlPc           4                  10        9   
...          ...         ...                 ...      ...   
15238    Guidare        3952                  10        9   
15239    Guidare        3953                  10        9   
15240    Guidare        3954                  10        9   
15241    Guidare        3955                  10        9   
15242    Guidare        3956                  10        9   

       AccelerometroXMedia  AccelerometroXDeviazioneStandard  \
0                 7.247556                          2.759148   
1                 7.746492                          2.466763   
2                 7.602547                          2.666870

## Bilanciamento dei Dataset ##

In [26]:
featureGlobalMaster = {}
featureGlobalMasterBilanciato = {}

# Itera attraverso i risultati della configurazione master
for utente, df_tot_master in risultati_configurazione_master.items():
    attivitaMaster = df_tot_master['Attivita']
    finestraIDMaster = df_tot_master['FinestraID']
    featuresMaster = df_tot_master.iloc[:, 4:]

    # Aggiungi la colonna 'Attivita' al dataframe delle features
    featuresMaster['Attivita'] = attivitaMaster
    featuresMaster['FinestraID'] = finestraIDMaster

    # Aggiungi i risultati della configurazione all'interno del dizionario dell'utente
    featureGlobalMaster[utente] = featuresMaster

# Itera attraverso il dizionario di utenti e dataframe
for utente, featuresMaster in featureGlobalMaster.items():
    print(f"Utente: {utente}")
    
    # Trova il numero minimo di occorrenze per ciascuna classe
    min_occorrenze_classe = featuresMaster['Attivita'].value_counts().min()

    # Tronca tutti i dataframe dell'utente al numero minimo di occorrenze
    featuresMaster_troncati = featuresMaster.groupby('Attivita').sample(n=min_occorrenze_classe, random_state=42)
    
    print(featuresMaster_troncati['Attivita'].value_counts())

    featureGlobalMasterBilanciato[utente] = featuresMaster_troncati

Utente: Giuseppe
Guidare      5420
StareAlPc    5420
Name: Attivita, dtype: int64


In [27]:
featureGlobalSlave = {}
featureGlobalSlaveBilanciato = {}

# Itera attraverso i risultati della configurazione slave
for utente, df_tot_slave in risultati_configurazione_slave.items():
    attivitaSlave = df_tot_slave['Attivita']
    finestraIDSlave = df_tot_slave['FinestraID']
    featuresSlave = df_tot_slave.iloc[:, 4:]

    # Aggiungi la colonna 'Attivita' al dataframe delle features
    featuresSlave['Attivita'] = attivitaSlave
    featuresSlave['FinestraID'] = finestraIDSlave

    # Aggiungi i risultati della configurazione all'interno del dizionario dell'utente
    featureGlobalSlave[utente] = featuresSlave

# Itera attraverso il dizionario di utenti e dataframe
for utente, featuresSlave in featureGlobalSlave.items():
    print(f"Utente: {utente}")
    
    # Trova il numero minimo di occorrenze per ciascuna classe
    min_occorrenze_classe = featuresSlave['Attivita'].value_counts().min()

    # Tronca tutti i dataframe dell'utente al numero minimo di occorrenze
    featuresSlave_troncati = featuresSlave.groupby('Attivita').sample(n=min_occorrenze_classe, random_state=42)
    
    print(featuresSlave_troncati['Attivita'].value_counts())

    featureGlobalSlaveBilanciato[utente] = featuresSlave_troncati

Utente: Giuseppe
Guidare      5421
StareAlPc    5421
Name: Attivita, dtype: int64


In [28]:
df_features_master_complex = pd.DataFrame()
    
# Concatenate the features for each user
for utente, featuresMaster_troncati in featureGlobalMasterBilanciato.items():
        
    df_features_master_complex = pd.concat([df_features_master_complex, featuresMaster_troncati], ignore_index=True)

print(df_features_master_complex.shape)

(10840, 37)


In [29]:
df_features_slave_complex = pd.DataFrame()
    
# Concatenate the features for each user
for utente, featuresSlave_troncati in featureGlobalSlaveBilanciato.items():
        
    df_features_slave_complex = pd.concat([df_features_slave_complex, featuresSlave_troncati], ignore_index=True)

print(df_features_slave_complex.shape)

(10842, 37)


In [33]:
# Esempio: Conteggio delle attività in df_features_master
activity_counts_master = df_features_master_complex['Attivita'].value_counts()

# Esempio: Conteggio delle attività nello slave DataFrame
activity_counts_slave = df_features_slave_complex['Attivita'].value_counts()

# Trova l'attività con il conteggio minore tra master e slave
min_activity_count = min(activity_counts_master.min(), activity_counts_slave.min())

# Filtra le attività nel master DataFrame in modo che ciascuna abbia lo stesso conteggio minimo
df_features_master_truncated_complex = df_features_master_complex.groupby('Attivita').sample(n=min_activity_count, random_state=42)

df_features_slave_truncated_complex = df_features_slave_complex.groupby('Attivita').sample(n=min_activity_count, random_state=42)

## Unione dei Dataset delle Features per dispositivo ##

In [36]:
features_test_final_master = pd.concat([features_test_master, df_features_master_truncated_complex], axis=0, ignore_index=True)
features_test_final_slave = pd.concat([features_test_slave, df_features_slave_truncated_complex], axis=0, ignore_index=True)

In [37]:
# Print the shape and value counts for each truncated dataset
print(f"Shape: {features_test_final_master.shape}")

# Print value counts for each activity
value_counts_per_activity = features_test_final_master['Attivita'].value_counts()
print("Value Counts per Activity:")
print(value_counts_per_activity)
print("\n")

# Print the shape and value counts for each truncated dataset
print(f"Shape: {features_test_final_slave.shape}")

# Print value counts for each activity
value_counts_per_activity = features_test_final_slave['Attivita'].value_counts()
print("Value Counts per Activity:")
print(value_counts_per_activity)
print("\n")

Shape: (17137, 37)
Value Counts per Activity:
Guidare      5420
StareAlPc    5420
Salire       1260
Camminata    1260
Fermo        1259
Corsa        1259
Scendere     1259
Name: Attivita, dtype: int64


Shape: (17137, 37)
Value Counts per Activity:
Guidare      5420
StareAlPc    5420
Salire       1260
Camminata    1260
Fermo        1259
Corsa        1259
Scendere     1259
Name: Attivita, dtype: int64




In [38]:
# Print the shape and value counts for each truncated dataset
print(f"Shape: {features_test_final_master.shape}")

# Print value counts for each activity
value_counts_per_activity = features_test_final_master['Attivita'].value_counts()
print("Value Counts per Activity:")
print(value_counts_per_activity)
print("\n")

# Print the shape and value counts for each truncated dataset
print(f"Shape: {features_test_final_slave.shape}")

# Print value counts for each activity
value_counts_per_activity = features_test_final_slave['Attivita'].value_counts()
print("Value Counts per Activity:")
print(value_counts_per_activity)
print("\n")

Shape: (17137, 37)
Value Counts per Activity:
Guidare      5420
StareAlPc    5420
Salire       1260
Camminata    1260
Fermo        1259
Corsa        1259
Scendere     1259
Name: Attivita, dtype: int64


Shape: (17137, 37)
Value Counts per Activity:
Guidare      5420
StareAlPc    5420
Salire       1260
Camminata    1260
Fermo        1259
Corsa        1259
Scendere     1259
Name: Attivita, dtype: int64




In [39]:
# Esempio: Conteggio delle attività in df_features_master
activity_counts_master = features_test_final_master['Attivita'].value_counts()

# Esempio: Conteggio delle attività nello slave DataFrame
activity_counts_slave = features_test_final_slave['Attivita'].value_counts()

# Trova l'attività con il conteggio minore tra master e slave
min_activity_count = min(activity_counts_master.min(), activity_counts_slave.min())

# Filtra le attività nel master DataFrame in modo che ciascuna abbia lo stesso conteggio minimo
features_master_test_ensamble = features_test_final_master.groupby('Attivita').sample(n=min_activity_count, random_state=42)

features_slave_test_ensamble = features_test_final_slave.groupby('Attivita').sample(n=min_activity_count, random_state=42)

In [40]:
print(features_master_test_ensamble['Attivita'].value_counts())

Camminata    1259
Corsa        1259
Fermo        1259
Guidare      1259
Salire       1259
Scendere     1259
StareAlPc    1259
Name: Attivita, dtype: int64


In [41]:
print(features_slave_test_ensamble['Attivita'].value_counts())

Camminata    1259
Corsa        1259
Fermo        1259
Guidare      1259
Salire       1259
Scendere     1259
StareAlPc    1259
Name: Attivita, dtype: int64


In [42]:
targets_test_master_ensamble = features_master_test_ensamble['Attivita']

targets_test_slave_enamble = features_slave_test_ensamble['Attivita']

In [43]:
listaAttivita = ["Fermo", "Camminata", "Corsa", "StareAlPc", "Salire", "Scendere", "Guidare"]

## K-Means Algorithm Parameters ##

In [44]:
kmeans_kwargs = {
    "init": "random",
    "n_init": "auto",
    "max_iter": 15000,
    "random_state": 42,
}

## Neural Network Models ##

In [45]:
def dnn_master(n_feature, n_classes, optimizer):
    model = Sequential()
    model.add(Dense(128, activation='relu', kernel_initializer='Identity', input_shape=(n_feature,)))
    model.add(Dense(128, activation='relu', kernel_initializer='Identity'))
    model.add(Dense(128, activation='relu', kernel_initializer='Identity'))
    model.add(Dense(n_classes, activation='softmax'))

    model.summary()
    return model

In [46]:
def dnn_slave(n_feature, n_classes, optimizer):
    model = Sequential()
    model.add(Dense(128, activation='relu', kernel_initializer='Identity', input_shape=(n_feature,)))
    model.add(Dense(128, activation='relu', kernel_initializer='Identity'))
    model.add(Dense(128, activation='relu', kernel_initializer='Identity'))
    model.add(Dense(n_classes, activation='softmax'))

    model.summary()
    return model

## Metriche ##

In [47]:
# tutti i valori riconosciuti correttamente, che si trovano dunque sulla diagonale principale della confusion matrix
def truePositive(confMatrix):
    return np.diag(confMatrix)

# l'insieme dei valori sulla stessa riga della label che stiamo attenzionando, escluso il relativo TP
def falsePositive(confMatrix):
    return confMatrix.sum(axis=0) - truePositive(confMatrix)

# l'insieme dei valori sulla stessa colonna della label che stiamo attenzionando, escluso il relativo TP
def falseNegative(confMatrix):
    return confMatrix.sum(axis=1) - truePositive(confMatrix)

def metricDataframe(cf, m1, m2, m3, m4, listaAttivita):
    metric1 = m1
    metric2 = m2
    metric3 = m3 
    metric4 = m4
    # Check if the confusion matrix has dimensions smaller than len(listaAttivita) x len(listaAttivita)
    if cf.shape != (len(listaAttivita), len(listaAttivita)):
        # If the dimensions are incorrect, set all metrics to 0 and error to 1
        global_metrics_df = pd.DataFrame({'Accuracy': 0, 'Macro_Precision': 0, 'Macro_Recall': 0, 'Macro_F1': 0, 'NumPredTot': metric1, 'NumPredMagSoglia': metric2, 'NumPredMinSoglia': metric3, 'PercentageErr': metric4, 'PercentageSucc': 1 - metric4}, index=['Values'])
    else:
        # Verifica se c'è almeno una riga o colonna con tutti zeri nella matrice di confusione
        if np.any(np.all(cf == 0, axis=1)) or np.any(np.all(cf == 0, axis=0)):
            # Tutte le metriche sono nulle se c'è almeno una riga o colonna con tutti zeri nella matrice di confusione
            global_metrics_df = pd.DataFrame({'Accuracy': 0, 'Macro_Precision': 0, 'Macro_Recall': 0, 'Macro_F1': 0, 'NumPredTot': metric1, 'NumPredMagSoglia': metric2, 'NumPredMinSoglia': metric3, 'PercentageErr': metric4, 'PercentageSucc': 1 - metric4}, index=['Values'])
        else:
            TP = truePositive(cf)
            FN = falseNegative(cf)
            FP = falsePositive(cf)

            accuracy = np.round(np.sum(TP) / np.sum(cf), 3)

            precision = np.round(TP / (TP + FP + 1e-10), 3)
            recall = np.round(TP / (TP + FN + 1e-10), 3)
            f1 = np.round((2 * TP) / (2 * TP + FP + FN + 1e-10), 3)

            global_metrics = {
                'Accuracy': accuracy,
                'Macro_Precision': np.round(np.mean(precision), 3),
                'Macro_Recall': np.round(np.mean(recall), 3),
                'Macro_F1': np.round(np.mean(f1), 3),
                'NumPredTot': metric1,
                'NumPredMagSoglia': metric2,
                'NumPredMinSoglia': metric3,
                'PercentageErr': metric4,
                'PercentageSucc': 1 - metric4,
            }

            # Creo un DataFrame con la colonna "Class" e le colonne "Precision", "Recall" e "F1_score" per ciascuna classe
            global_metrics_df = pd.DataFrame(global_metrics, index=['Values'])

            # Converti le colonne in formato numerico per il DataFrame
            global_metrics_df[["Accuracy", "Macro_Precision", "Macro_Recall", "Macro_F1", "NumPredTot", "NumPredMagSoglia", "NumPredMinSoglia", "PercentageErr", "PercentageSucc"]] = global_metrics_df[["Accuracy", "Macro_Precision", "Macro_Recall", "Macro_F1", "NumPredTot", "NumPredMagSoglia", "NumPredMinSoglia", "PercentageErr", "PercentageSucc"]].apply(pd.to_numeric, errors='coerce')

    return global_metrics_df

## Data Fusion Algorithm ##

In [ ]:
import pandas as pd
from collections import Counter

# Numero di fold per la cross-validation
num_folds = 10

skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for k_master in range(10,51,10):
    for k_slave in range(10,51,10):
        
        chiave_configurazione = f"{k_master}_{k_slave}"

        features_split_master_df = features_train_master.iloc[:, :-2]
        features_split_slave_df = features_train_slave.iloc[:, :-2]

        kmeans_master = KMeans(n_clusters=k_master, **kmeans_kwargs)
        kmeans_data_master = kmeans_master.fit(features_split_master_df)
        cluster_labels_master = kmeans_data_master.labels_
    
        print(f"K: {chiave_configurazione}")
    
        # Store both cluster labels and data in a dictionary with the activity as key
        features_split_master_df['Cluster'] = cluster_labels_master
    
        # Creazione del dizionario features_master_cluster
        features_master_cluster = {}

        # Assegna etichette letterali ai cluster (A, B, C, ...)
        label_mapping = {i: chr(120481 + i) for i in range(k_master)}

        # Itera sui cluster e assegna le features corrispondenti
        for cluster_label in range(k_master):
            cluster_name_master = label_mapping[cluster_label]
            cluster_features_master = features_split_master_df[features_split_master_df['Cluster'] == cluster_label].drop(columns=['Cluster'])
            cluster_features_master['Labels'] = cluster_name_master
            features_master_cluster[cluster_name_master] = cluster_features_master
        
        # Concatena tutti i DataFrame nel dizionario in un unico DataFrame
        df_completo_master = pd.concat(features_master_cluster.values())

        # Resetta gli indici per avere un indice numerico
        df_completo_master = df_completo_master.reset_index(level=0, drop=True)
    
        labels_master = df_completo_master['Labels']

        # Creazione di un LabelEncoder per convertire le etichette stringhe in numeri durante l'addestramento
        label_encoder_master = LabelEncoder()

        # Fit del LabelEncoder alle etichette di classe e trasformazione delle etichette in formato numerico
        target_onehot_master = label_encoder_master.fit_transform(labels_master)
        
        # Trasformo in tensore
        features_master = df_completo_master.iloc[:, :-1]
        features_master = features_master.values

        # Supponendo che 'features_train', 'labels_train', 'features_test', 'labels_test' siano i tuoi dati
        model_master = dnn_master(features_master.shape[1], len(labels_master.unique()), optimizer='adam')
            
        # Compila il modello con l'accuracy come metrica
        model_master.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        print("-------------- Addestramento modello MASTER ------------------")
        # Addestra il modello
        model_master.fit(features_master, target_onehot_master, epochs=25, batch_size=32)

        # Initialize an empty dictionary to store results
        result_dict_master = {}

        for attivita in listaAttivita:
            
            df_temp_master = pd.DataFrame()

            for _ in range(20): 
            
                features_master_test_attivita = features_master_test_ensamble[features_master_test_ensamble['Attivita'] == attivita]
                features_master_test_attivita = features_master_test_attivita.iloc[:, :-2]
                features_master_test_attivita = features_master_test_attivita.values

                # Esegui la predizione
                y_prediction_master_semplici = model_master.predict(features_master_test_attivita)

                # Decodifica delle predizioni in formato stringa utilizzando il LabelEncoder
                predictions_string_master_semplici = label_encoder_master.inverse_transform(y_prediction_master_semplici.argmax(axis=1))
                
                frame_pred_master = pd.DataFrame({'PriorM': predictions_string_master_semplici, 'AttivitaM': attivita})
                df_temp_master = pd.concat([df_temp_master, frame_pred_master], axis=0)

            # Add the result to the dictionary
            result_dict_master[attivita] = df_temp_master

        kmeans_slave = KMeans(n_clusters=k_slave, **kmeans_kwargs)
        kmeans_data_slave = kmeans_slave.fit(features_split_slave_df)
        cluster_labels_slave = kmeans_data_slave.labels_
    
        # Store both cluster labels and data in a dictionary with the activity as key
        features_split_slave_df['Cluster'] = cluster_labels_slave
    
        # Creazione del dizionario features_slave_cluster
        features_slave_cluster = {}

        # Assegna etichette letterali ai cluster (A, B, C, ...)
        label_mapping = {i: chr(120481 + i) for i in range(k_slave)}

        # Itera sui cluster e assegna le features corrispondenti
        for cluster_label in range(k_slave):
            cluster_name_slave = label_mapping[cluster_label]
            cluster_features_slave = features_split_slave_df[features_split_slave_df['Cluster'] == cluster_label].drop(columns=['Cluster'])
            cluster_features_slave['Labels'] = cluster_name_slave
            features_slave_cluster[cluster_name_slave] = cluster_features_slave
        
        # Concatena tutti i DataFrame nel dizionario in un unico DataFrame
        df_completo_slave = pd.concat(features_slave_cluster.values())

        # Resetta gli indici per avere un indice numerico
        df_completo_slave = df_completo_slave.reset_index(level=0, drop=True)
    
        labels_slave = df_completo_slave['Labels']

        # Creazione di un LabelEncoder per convertire le etichette stringhe in numeri durante l'addestramento
        label_encoder_slave = LabelEncoder()

        # Fit del LabelEncoder alle etichette di classe e trasformazione delle etichette in formato numerico
        target_onehot_slave = label_encoder_slave.fit_transform(labels_slave)
        
        # Estrai le features e i target come array NumPy
        features_slave = df_completo_slave.iloc[:, :-1]
        features_slave = features_slave.values

        # Supponendo che 'features_train', 'labels_train', 'features_test', 'labels_test' siano i tuoi dati
        model_slave = dnn_slave(features_slave.shape[1], len(labels_slave.unique()), optimizer='adam')
            
        # Compila il modello con l'accuracy come metrica
        model_slave.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        print("-------------- Addestramento modello slave ------------------")
        # Addestra il modello
        model_slave.fit(features_slave, target_onehot_slave, epochs=25, batch_size=32)

        # Initialize an empty dictionary to store results
        result_dict_slave = {}

        for attivita in listaAttivita:
            
            df_temp_slave = pd.DataFrame()
            
            for _ in range(20): 
                
                features_slave_test_attivita = features_slave_test_ensamble[features_slave_test_ensamble['Attivita'] == attivita]
                features_slave_test_attivita = features_slave_test_attivita.iloc[:, :-2]
                features_slave_test_attivita = features_slave_test_attivita.values

                # Esegui la predizione
                y_prediction_slave_semplici = model_slave.predict(features_slave_test_attivita)

                # Decodifica delle predizioni in formato stringa utilizzando il LabelEncoder
                predictions_string_slave_semplici = label_encoder_slave.inverse_transform(y_prediction_slave_semplici.argmax(axis=1))
                
                frame_pred_slave = pd.DataFrame({'PriorS': predictions_string_slave_semplici, 'Attivita': attivita})
                
                df_temp_slave = pd.concat([df_temp_slave, frame_pred_slave], axis=0)
            
            # Add the result to the dictionary
            result_dict_slave[attivita] = df_temp_slave
            
        df_prior_tot = pd.DataFrame()

        for attivita, df_temp_master in result_dict_master.items():
            df_temp_slave = result_dict_slave.get(attivita, pd.DataFrame()) 

            # Adjust the concatenation based on your DataFrame structure
            df_prior_temp = pd.concat([df_temp_master, df_temp_slave], axis=1).drop(columns='AttivitaM')

            df_prior_tot = pd.concat([df_prior_tot, df_prior_temp], axis=0)
            
        df_prior_tot = df_prior_tot.reset_index(drop=True)
        
        print(df_prior_tot.shape)
        
        print("Dataset delle Predizioni")

        labels_prior = df_prior_tot['Attivita']
        df_prior_tot = df_prior_tot.iloc[:, :-1]
        
        for fold, (train_index, test_index) in enumerate(skf.split(df_prior_tot, labels_prior)):
        
            print("-" * 50)
            print(f'Fold {fold + 1} - Train set: {len(train_index)}, Test set: {len(test_index)}')
            print("-" * 50)
            
            folds_train, folds_test = df_prior_tot.iloc[train_index], df_prior_tot.iloc[test_index]
            targets_train, targets_test = labels_prior.iloc[train_index], labels_prior.iloc[test_index]
            
            folds_train['Attivita'] = targets_train
            folds_test['Attivita'] = targets_test
            
            # Aggiungi una colonna 'Index' a folds_test con indici numerati progressivamente
            folds_test['Index'] = range(len(targets_test))
            
            #print(folds_test)
            
            folds_test_nolabel = folds_test.iloc[:, :-2]
        
            edges = [('PriorM', 'Attivita'), ('PriorS', 'Attivita')]
        
            DAG = bn.make_DAG(edges)

            model_bnn = bn.parameter_learning.fit(DAG, folds_train, methodtype='maximumlikelihood')

            # Esegui la predizione
            out = bn.predict(model_bnn, folds_test_nolabel, variables=['Attivita'])
            
            print(out)

            # Aggiungi una colonna 'Index' al DataFrame risultante da bn.predict
            out['Index'] = range(len(targets_test))

            # Creazione della matrice di confusione solo per le etichette predette e reali con gli stessi indici
            for threshold in np.arange(0.60, 1.01, 0.10):
                threshold = round(threshold, 2)
                pred_label_list = []
                pred_label_list_sconosciute = []

                for index, row in out.iterrows():
                    # Controllo se la probabilità ('p') è maggiore o uguale a 0.6
                    if row['p'] >= threshold:
                        pred_label = row['Attivita']
                        pred_label_list.append((index, pred_label))  # Salvo l'etichetta e l'indice originale
                    else:
                        # Se la probabilità è inferiore alla soglia, aggiungi l'etichetta a pred_label_list_sconosciute
                        pred_label_sconosciuta = row['Attivita']
                        pred_label_list_sconosciute.append((index, pred_label_sconosciuta))

                # Creo un DataFrame per le etichette predette mantenendo l'indice originale
                pred_df = pd.DataFrame(pred_label_list, columns=['Index', 'Attivita'])
                
                # Se non ci sono attività predette, crea una matrice di confusione vuota con tutti gli elementi a 0
                if pred_df.empty:
                    num_pred_correct = 0
                    num_pred_sconosciute = folds_test.shape[0]
                    num_tot = folds_test.shape[0]
                    percentage = 1.0

                    # Crea una matrice di confusione con tutti gli elementi a 0
                    cf = np.zeros((len(listaAttivita), len(listaAttivita)))
                    
                    plt.figure(figsize=(15, 10))
                    sns.heatmap(cf, annot=True, fmt='.2%', cmap='gray', xticklabels=y_true, yticklabels=y_pred)
                    plt.title(f'Confusion Matrix - Precisione sulle predizioni con soglia: {threshold}')
                    plt.xlabel('Predicted values')
                    plt.ylabel('True values')

                    # Crea il percorso della cartella e della sottodirectory
                    folder_path_matrix = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Matrix",)
                    os.makedirs(folder_path_matrix, exist_ok=True)

                    # Salva la matrice di confusione come immagine nella sottodirectory
                    image_name = f"ConfusionMatrixComplexFusionBayesian_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}.pdf"
                    image_path = os.path.join(folder_path_matrix, image_name)
                    plt.savefig(image_path, bbox_inches='tight')
                        
                    folder_path_csv_global = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Csv", "Global")
                    os.makedirs(folder_path_csv_global, exist_ok=True)
                        
                    csv_name_global = f"ConfusionMatrixComplexFusionBayesian_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}_Global.csv"
                    csv_path_global = os.path.join(folder_path_csv_global, csv_name_global)
                    
                    folder_path_csv_pred = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Csv", "PredizioniTotali")
                    os.makedirs(folder_path_csv_pred, exist_ok=True)
                    
                    csv_name_pred = f"PredizioniTotali_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}.csv"
                    csv_path_pred = os.path.join(folder_path_csv_pred, csv_name_pred)
                    
                    # Salvataggio Predizioni
                    out.to_csv(csv_path_pred)

                    # Chiamata a metricDataframe
                    metrics_df = metricDataframe(cf, num_tot, num_pred_correct, num_pred_sconosciute, percentage, listaAttivita)

                    metrics_df.to_csv(csv_path_global)

                     # Stampo il numero totale di attività predette e sconosciute
                    print("-" * 50)
                    # Stampa il numero totale di attività da predire
                    print(f"Numero di attività totali da predire: {num_tot}")
                    print(f"Numero di attività totali predette: {num_pred_correct}")
                    print(f"Numero di attività totali sconosciute: {num_pred_sconosciute}")
                    print(f"Percentuale di predizioni incerte sul totale: {percentage * 100:.2f}%")
                    print("-" * 50)
                    
                    plt.show()
                else:
                    # Creo un DataFrame per le etichette sconosciute mantenendo l'indice originale
                    sconosciute_df = pd.DataFrame(pred_label_list_sconosciute, columns=['Index', 'Attivita'])

                    # Inizializza un DataFrame vuoto per le etichette corrispondenti
                    matched_labels_df = pd.DataFrame(columns=['Index', 'Attivita_pred'])

                    # Itera sugli indici di pred_df
                    for index, row in pred_df.iterrows():
                        # Verifica se l'indice è presente in folds_test
                        if row['Index'] in folds_test['Index'].values:
                            # Ottieni la corrispondente etichetta reale da folds_test
                            true_label = folds_test.loc[folds_test['Index'] == row['Index'], 'Attivita'].values[0]
                            # Aggiungi l'indice e l'etichetta predetta al DataFrame matched_labels_df
                            matched_labels_df = matched_labels_df.append({'Index': row['Index'], 'Attivita_pred': row['Attivita'], 'Attivita_true': true_label}, ignore_index=True)
                    
                    y_true = matched_labels_df['Attivita_true'].values
                    y_pred = matched_labels_df['Attivita_pred'].values
                    
                    num_pred_correct = pred_df['Attivita'].count()
                    num_pred_sconosciute = sconosciute_df['Attivita'].count()
                    num_tot = folds_test.shape[0]
                    percentage = (sconosciute_df['Attivita'].count() / folds_test.shape[0])
                    
                    # Stampo il numero totale di attività predette e sconosciute
                    print("-" * 50)
                    # Stampa il numero totale di attività da predire
                    print(f"Numero di attività totali da predire: {num_tot}")
                    print(f"Numero di attività totali predette: {num_pred_correct}")
                    print(f"Numero di attività totali sconosciute: {num_pred_sconosciute}")
                    print(f"Percentuale di predizioni incerte sul totale: {percentage * 100:.2f}%")
                    print("-" * 50)

                    # Creazione della matrice di confusione solo per le etichette predette e reali con gli stessi indici
                    cf = confusion_matrix(y_true, y_pred, normalize='true')

                    plt.figure(figsize=(15, 10))
                    sns.heatmap(cf, annot=True, fmt='.2%', cmap='gray', xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
                    plt.title(f'Confusion Matrix - Precisione sulle predizioni con soglia: {threshold}')
                    plt.xlabel('Predicted values')
                    plt.ylabel('True values')

                    # Crea il percorso della cartella e della sottodirectory
                    folder_path_matrix = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Matrix",)
                    os.makedirs(folder_path_matrix, exist_ok=True)

                    # Salva la matrice di confusione come immagine nella sottodirectory
                    image_name = f"ConfusionMatrixComplexFusionBayesian_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}.pdf"
                    image_path = os.path.join(folder_path_matrix, image_name)
                    plt.savefig(image_path, bbox_inches='tight')
                        
                    folder_path_csv_global = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Csv", "Global")
                    os.makedirs(folder_path_csv_global, exist_ok=True)
                        
                    csv_name_global = f"ConfusionMatrixComplexFusionBayesian_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}_Global.csv"
                    csv_path_global = os.path.join(folder_path_csv_global, csv_name_global)
                    
                    folder_path_csv_pred = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Csv", "PredizioniTotali")
                    os.makedirs(folder_path_csv_pred, exist_ok=True)
                    
                    csv_name_pred = f"PredizioniTotali_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}.csv"
                    csv_path_pred = os.path.join(folder_path_csv_pred, csv_name_pred)
                    
                    folder_path_csv_pred = os.path.join("ConfusionMatrixComplexFusionBayesian", chiave_configurazione, str(threshold), "Csv", "MatchPredizioni")
                    os.makedirs(folder_path_csv_pred, exist_ok=True)
                    
                    csv_name_pred_match = f"MatchPredizioni_{chiave_configurazione}_Soglia_{str(threshold)}_Fold_{fold + 1}.csv"
                    csv_path_pred_match = os.path.join(folder_path_csv_pred, csv_name_pred)
                    
                    # Salvataggio Predizioni
                    out.to_csv(csv_path_pred)
                    
                    matched_labels_df.to_csv(csv_path_pred_match)

                    # Chiamata a metricDataframe
                    metrics_df = metricDataframe(cf, num_tot, num_pred_correct, num_pred_sconosciute, percentage, listaAttivita)

                    metrics_df.to_csv(csv_path_global)

                    plt.show()